In [1]:
# audio libraries
import librosa
import librosa.display as lplt
import IPython

# import matplotlib to be able to display graphs
import matplotlib.pyplot as plt

# transform .wav into .csv
import csv
import os
import numpy as np
import pandas as pd

# preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import joblib

# model
import keras
import tensorflow as tf
from tensorflow.keras.layers import Normalization
import datetime, os
%load_ext tensorboard
from tensorflow.keras.models import Sequential

In [2]:
target_sample_rate = 40000

def resample_audio(file_path, target_sr=target_sample_rate):
    # Загрузка аудиофайла с его исходной частотой дискретизации
    y, sr = librosa.load(file_path, mono = True, duration = 30)
    
    # Если исходная частота не совпадает с целевой, пересэмплируем
    if sr != target_sr:
        print(f"Файл {file_path} имеет частоту {sr} Гц, пересэмплируем до {target_sr} Гц.")
        y = librosa.resample(y, orig_sr=sr, target_sr=target_sr)
    else:
        print(f"Файл {file_path} уже имеет целевую частоту {target_sr} Гц.")
    
    return y, target_sr

In [3]:
#AtlanticSpottedDolphin BeardedSeal Beluga_WhiteWhale BlueWhale BottlenoseDolphin Boutu_AmazonRiverDolphin BowheadWhale ClymeneDolphin \
#Commerson'sDolphin CommonDolphin Dall'sPorpoise DuskyDolphin FalseKillerWhale Fin_FinbackWhale FinlessPorpoise Fraser'sDolphin Grampus_Risso'sDolphin \
#GraySeal GrayWhale HarborPorpoise HarbourSeal HarpSeal Heaviside'sDolphin HoodedSeal HumpbackWhale IrawaddyDolphin JuanFernandezFurSeal KillerWhale \
#LeopardSeal Long_FinnedPilotWhale LongBeaked(Pacific)CommonDolphin MelonHeadedWhale MinkeWhale Narwhal NewZealandFurSeal NorthernRightWhale \
#PantropicalSpottedDolphin RibbonSeal RingedSeal RossSeal Rough_ToothedDolphin SeaOtter Short_Finned(Pacific)PilotWhale SouthernRightWhale SpermWhale

In [4]:
header = "length chroma_stft_mean chroma_stft_var rms_mean rms_var spectral_centroid_mean spectral_centroid_var spectral_bandwidth_mean \
        spectral_bandwidth_var rolloff_mean rolloff_var zero_crossing_rate_mean zero_crossing_rate_var harmony_mean harmony_var perceptr_mean \
        perceptr_var tempo mfcc1_mean mfcc1_var mfcc2_mean mfcc2_var mfcc3_mean mfcc3_var mfcc4_mean mfcc4_var label".split()
file = open("/kaggle/working/data.csv", "w", newline = "")
with file:
    writer = csv.writer(file)
    writer.writerow(header)
marine_mammals = "AtlanticSpottedDolphin BeardedSeal Beluga_WhiteWhale BottlenoseDolphin BowheadWhale ClymeneDolphin \
        CommonDolphin FalseKillerWhale Fin_FinbackWhale Fraser'sDolphin \
        HumpbackWhale KillerWhale MelonHeadedWhale NorthernRightWhale \
        PantropicalSpottedDolphin Short_Finned(Pacific)PilotWhale SpermWhale".split()
for animal in marine_mammals:

    for filename in os.listdir(f"/kaggle/input/best-of-watkins-marine-mammal-sound-database/data/{animal}/"):

        sound_name = f"/kaggle/input/best-of-watkins-marine-mammal-sound-database/data/{animal}/{filename}"
        y, sr = resample_audio(sound_name)
        chroma_stft = librosa.feature.chroma_stft(y = y, sr = sr)
        rmse = librosa.feature.rms(y = y)
        spec_cent = librosa.feature.spectral_centroid(y = y, sr = sr)
        spec_bw = librosa.feature.spectral_bandwidth(y = y, sr = sr)
        rolloff = librosa.feature.spectral_rolloff(y = y, sr = sr)
        zcr = librosa.feature.zero_crossing_rate(y)
        mfcc = librosa.feature.mfcc(y = y, sr = sr)
        to_append = f'{np.mean(chroma_stft)} {np.mean(rmse)} {np.mean(spec_cent)} {np.mean(spec_bw)} {np.mean(rolloff)} {np.mean(zcr)}'

        for e in mfcc:
            to_append += f' {np.mean(e)}'

        to_append += f' {animal}'
        file = open('/kaggle/working/data.csv', 'a', newline = '')

        with file:
            writer = csv.writer(file)
            writer.writerow(to_append.split())

Файл /kaggle/input/best-of-watkins-marine-mammal-sound-database/data/AtlanticSpottedDolphin/6102601F.wav имеет частоту 22050 Гц, пересэмплируем до 40000 Гц.
Файл /kaggle/input/best-of-watkins-marine-mammal-sound-database/data/AtlanticSpottedDolphin/6102601B.wav имеет частоту 22050 Гц, пересэмплируем до 40000 Гц.
Файл /kaggle/input/best-of-watkins-marine-mammal-sound-database/data/AtlanticSpottedDolphin/6102602S.wav имеет частоту 22050 Гц, пересэмплируем до 40000 Гц.
Файл /kaggle/input/best-of-watkins-marine-mammal-sound-database/data/AtlanticSpottedDolphin/6102604N.wav имеет частоту 22050 Гц, пересэмплируем до 40000 Гц.
Файл /kaggle/input/best-of-watkins-marine-mammal-sound-database/data/AtlanticSpottedDolphin/6102600L.wav имеет частоту 22050 Гц, пересэмплируем до 40000 Гц.
Файл /kaggle/input/best-of-watkins-marine-mammal-sound-database/data/AtlanticSpottedDolphin/61026016.wav имеет частоту 22050 Гц, пересэмплируем до 40000 Гц.
Файл /kaggle/input/best-of-watkins-marine-mammal-sound-dat

/opt/conda/lib/python3.10/site-packages/librosa/core/pitch.py:101: UserWarning: Trying to estimate tuning from empty frequency set.
  return pitch_tuning(


Файл /kaggle/input/best-of-watkins-marine-mammal-sound-database/data/AtlanticSpottedDolphin/61026047.wav имеет частоту 22050 Гц, пересэмплируем до 40000 Гц.
Файл /kaggle/input/best-of-watkins-marine-mammal-sound-database/data/AtlanticSpottedDolphin/6102501B.wav имеет частоту 22050 Гц, пересэмплируем до 40000 Гц.
Файл /kaggle/input/best-of-watkins-marine-mammal-sound-database/data/AtlanticSpottedDolphin/6102600W.wav имеет частоту 22050 Гц, пересэмплируем до 40000 Гц.
Файл /kaggle/input/best-of-watkins-marine-mammal-sound-database/data/AtlanticSpottedDolphin/6102602A.wav имеет частоту 22050 Гц, пересэмплируем до 40000 Гц.
Файл /kaggle/input/best-of-watkins-marine-mammal-sound-database/data/AtlanticSpottedDolphin/6102501D.wav имеет частоту 22050 Гц, пересэмплируем до 40000 Гц.
Файл /kaggle/input/best-of-watkins-marine-mammal-sound-database/data/AtlanticSpottedDolphin/6102600O.wav имеет частоту 22050 Гц, пересэмплируем до 40000 Гц.
Файл /kaggle/input/best-of-watkins-marine-mammal-sound-dat

/opt/conda/lib/python3.10/site-packages/librosa/core/spectrum.py:266: UserWarning: n_fft=2048 is too large for input signal of length=1036
  warnings.warn(


Файл /kaggle/input/best-of-watkins-marine-mammal-sound-database/data/SpermWhale/8800201B.wav имеет частоту 22050 Гц, пересэмплируем до 40000 Гц.
Файл /kaggle/input/best-of-watkins-marine-mammal-sound-database/data/SpermWhale/90043003.wav имеет частоту 22050 Гц, пересэмплируем до 40000 Гц.
Файл /kaggle/input/best-of-watkins-marine-mammal-sound-database/data/SpermWhale/8500901R.wav имеет частоту 22050 Гц, пересэмплируем до 40000 Гц.
Файл /kaggle/input/best-of-watkins-marine-mammal-sound-database/data/SpermWhale/9551900C.wav имеет частоту 22050 Гц, пересэмплируем до 40000 Гц.
Файл /kaggle/input/best-of-watkins-marine-mammal-sound-database/data/SpermWhale/9004600O.wav имеет частоту 22050 Гц, пересэмплируем до 40000 Гц.
Файл /kaggle/input/best-of-watkins-marine-mammal-sound-database/data/SpermWhale/94201007.wav имеет частоту 22050 Гц, пересэмплируем до 40000 Гц.
Файл /kaggle/input/best-of-watkins-marine-mammal-sound-database/data/SpermWhale/7500502B.wav имеет частоту 22050 Гц, пересэмплируе

...

Файл /kaggle/input/best-of-watkins-marine-mammal-sound-database/data/SpermWhale/8800202P.wav имеет частоту 22050 Гц, пересэмплируем до 40000 Гц.
Файл /kaggle/input/best-of-watkins-marine-mammal-sound-database/data/SpermWhale/91003010.wav имеет частоту 22050 Гц, пересэмплируем до 40000 Гц.
Файл /kaggle/input/best-of-watkins-marine-mammal-sound-database/data/SpermWhale/85003002.wav имеет частоту 22050 Гц, пересэмплируем до 40000 Гц.
Файл /kaggle/input/best-of-watkins-marine-mammal-sound-database/data/SpermWhale/7501100F.wav имеет частоту 22050 Гц, пересэмплируем до 40000 Гц.
Файл /kaggle/input/best-of-watkins-marine-mammal-sound-database/data/SpermWhale/93584001.wav имеет частоту 22050 Гц, пересэмплируем до 40000 Гц.
Файл /kaggle/input/best-of-watkins-marine-mammal-sound-database/data/SpermWhale/85002017.wav имеет частоту 22050 Гц, пересэмплируем до 40000 Гц.
Файл /kaggle/input/best-of-watkins-marine-mammal-sound-database/data/SpermWhale/7500600D.wav имеет частоту 22050 Гц, пересэмплируе

/opt/conda/lib/python3.10/site-packages/librosa/core/spectrum.py:266: UserWarning: n_fft=2048 is too large for input signal of length=1151
  warnings.warn(


Файл /kaggle/input/best-of-watkins-marine-mammal-sound-database/data/SpermWhale/93591002.wav имеет частоту 22050 Гц, пересэмплируем до 40000 Гц.
Файл /kaggle/input/best-of-watkins-marine-mammal-sound-database/data/SpermWhale/8500901A.wav имеет частоту 22050 Гц, пересэмплируем до 40000 Гц.
Файл /kaggle/input/best-of-watkins-marine-mammal-sound-database/data/SpermWhale/88002026.wav имеет частоту 22050 Гц, пересэмплируем до 40000 Гц.
Файл /kaggle/input/best-of-watkins-marine-mammal-sound-database/data/SpermWhale/75012003.wav имеет частоту 22050 Гц, пересэмплируем до 40000 Гц.
Файл /kaggle/input/best-of-watkins-marine-mammal-sound-database/data/SpermWhale/85002015.wav имеет частоту 22050 Гц, пересэмплируем до 40000 Гц.
Файл /kaggle/input/best-of-watkins-marine-mammal-sound-database/data/SpermWhale/8103800T.wav имеет частоту 22050 Гц, пересэмплируем до 40000 Гц.
Файл /kaggle/input/best-of-watkins-marine-mammal-sound-database/data/SpermWhale/7702801A.wav имеет частоту 22050 Гц, пересэмплируе

/opt/conda/lib/python3.10/site-packages/librosa/core/spectrum.py:266: UserWarning: n_fft=2048 is too large for input signal of length=1033
  warnings.warn(


Файл /kaggle/input/best-of-watkins-marine-mammal-sound-database/data/SpermWhale/72009001.wav имеет частоту 22050 Гц, пересэмплируем до 40000 Гц.
Файл /kaggle/input/best-of-watkins-marine-mammal-sound-database/data/SpermWhale/95516017.wav имеет частоту 22050 Гц, пересэмплируем до 40000 Гц.
Файл /kaggle/input/best-of-watkins-marine-mammal-sound-database/data/SpermWhale/7501204Q.wav имеет частоту 22050 Гц, пересэмплируем до 40000 Гц.
Файл /kaggle/input/best-of-watkins-marine-mammal-sound-database/data/SpermWhale/7200900O.wav имеет частоту 22050 Гц, пересэмплируем до 40000 Гц.
Файл /kaggle/input/best-of-watkins-marine-mammal-sound-database/data/SpermWhale/85002012.wav имеет частоту 22050 Гц, пересэмплируем до 40000 Гц.
Файл /kaggle/input/best-of-watkins-marine-mammal-sound-database/data/SpermWhale/7501101O.wav имеет частоту 22050 Гц, пересэмплируем до 40000 Гц.
Файл /kaggle/input/best-of-watkins-marine-mammal-sound-database/data/SpermWhale/90009005.wav имеет частоту 22050 Гц, пересэмплируе

In [5]:
df = pd.read_csv('/kaggle/working/data.csv')
df.shape
os.makedirs(f'/kaggle/working/model_versions', exist_ok=True)
df.head()

,length,chroma_stft_mean,chroma_stft_var,rms_mean,rms_var,spectral_centroid_mean,spectral_centroid_var,spectral_bandwidth_mean,spectral_bandwidth_var,rolloff_mean,...,tempo,mfcc1_mean,mfcc1_var,mfcc2_mean,mfcc2_var,mfcc3_mean,mfcc3_var,mfcc4_mean,mfcc4_var,label
0,0.563931,0.058410,4657.979661,2764.597756,7766.073258,0.213951,-291.571350,123.517998,-75.955879,117.515511,...,-14.099276,14.732727,6.621064,-1.363451,20.940311,-7.665897,16.509853,-5.118043,5.581205,AtlanticSpottedDolphin
1,0.593863,0.034768,4652.132998,2950.589824,7959.352889,0.200730,-308.498291,123.499184,-71.451332,113.824623,...,-11.257586,14.537269,7.576907,-1.231695,16.605612,-8.450237,15.084641,-4.486030,5.225972,AtlanticSpottedDolphin
2,0.592817,0.019791,4945.657779,3120.128652,8411.668347,0.217149,-349.475708,130.631332,-75.262291,117.976921,...,-13.721874,20.523043,2.890947,-1.006940,20.200640,-13.066253,19.526924,-7.847659,3.457018,AtlanticSpottedDolphin
3,0.536554,0.023658,4477.150890,2964.537123,7862.966653,0.188326,-348.420990,115.605217,-51.604053,98.001266,...,-7.833398,12.752177,8.680356,-3.040437,15.008431,-4.866544,11.887104,-2.358061,6.178308,AtlanticSpottedDolphin
4,0.536654,0.021080,5186.379777,2996.924034,8380.664062,0.262402,-339.538757,128.845413,-83.089813,120.647278,...,-16.329111,17.696386,6.229254,-5.042972,18.067245,-11.200329,14.994453,-8.728179,5.471106,AtlanticSpottedDolphin


In [6]:
# Feature extraction
class_list = df.iloc[:,-1]
print(class_list)
encoder = LabelEncoder()
y = encoder.fit_transform(class_list)
print("y: ", y)
input_parameters = df.iloc[:, 0:26]
scaler = StandardScaler()
X = scaler.fit_transform(np.array(input_parameters))
print("X:", X)

0        AtlanticSpottedDolphin
1        AtlanticSpottedDolphin
2        AtlanticSpottedDolphin
3        AtlanticSpottedDolphin
4        AtlanticSpottedDolphin
                  ...          
10602                SpermWhale
10603                SpermWhale
10604                SpermWhale
10605                SpermWhale
10606                SpermWhale
Name: label, Length: 10607, dtype: object
y:  [ 0  0  0 ... 16 16 16]
X: [[ 0.11643141  0.67420618  0.87701573 ...  0.64106911  0.03314103
   0.10594846]
 [ 0.32607314  0.04526751  0.87351419 ...  0.4721724   0.14177928
   0.03540919]
 [ 0.31874582 -0.35314292  1.04930482 ...  0.99861103 -0.43605908
  -0.31585499]
 ...
 [ 0.22947296 -0.56220213  0.69701619 ... -0.12421016 -1.31549655
  -0.24670013]
 [-1.45479989  0.07718401  0.90016406 ...  0.55025839 -0.28937907
  -0.4034333 ]
 [ 0.22332039  0.0562275   0.33558326 ...  0.38749541 -0.41266178
   0.59496265]]


In [7]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size = 0.2)
#normalizer = Normalization(axis=-1)
#normalizer.adapt(X_train)

In [8]:
#Setup model
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(512, activation = 'relu', input_shape = (X_train.shape[1],)),
    tf.keras.layers.Dropout(0.2),
    
    tf.keras.layers.Dense(256, activation = 'relu'),
    keras.layers.Dropout(0.2),
    
    tf.keras.layers.Dense(128, activation = 'relu'),
    tf.keras.layers.Dropout(0.2),
    
    tf.keras.layers.Dense(64, activation = 'relu'),
    tf.keras.layers.Dropout(0.2),
    
    tf.keras.layers.Dense(45, activation = 'softmax'),
])

print(model.summary())

/opt/conda/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 512)            │        13,824 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 256)            │       131,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 45)             │         2,925 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 189,229 (739.18 KB)

 Trainable params: 189,229 (739.18 KB)

 Non-trainable params: 0 (0.00 B)

None


In [9]:
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

def trainModel(model,epochs, optimizer):
    batch_size = 128
    model.compile(optimizer = optimizer, loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])
    return model.fit(X_train, y_train, validation_data = (X_val, y_val), epochs = epochs, batch_size = batch_size, callbacks=[tensorboard_callback])

In [10]:
#%tensorboard --logdir logs
model_history = trainModel(model = model, epochs = 100, optimizer = 'adam')

Epoch 1/100
67/67 ━━━━━━━━━━━━━━━━━━━━ 3s 12ms/step - accuracy: 0.3369 - loss: 2.5839 - val_accuracy: 0.6701 - val_loss: 1.0020
Epoch 2/100
67/67 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.6744 - loss: 1.0374 - val_accuracy: 0.7917 - val_loss: 0.6790
Epoch 3/100
67/67 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.7673 - loss: 0.7295 - val_accuracy: 0.8360 - val_loss: 0.5194
Epoch 4/100
67/67 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.8217 - loss: 0.5756 - val_accuracy: 0.8779 - val_loss: 0.4334
Epoch 5/100
67/67 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.8364 - loss: 0.5024 - val_accuracy: 0.8921 - val_loss: 0.3701
Epoch 6/100
67/67 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.8715 - loss: 0.4078 - val_accuracy: 0.9095 - val_loss: 0.3331
Epoch 7/100
67/67 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.8851 - loss: 0.3590 - val_accuracy: 0.9161 - val_loss: 0.2904
Epoch 8/100
67/67 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.9039 - loss: 0.3131 - val_accuracy: 0.9171 - 

In [11]:
#os.makedirs(f'/kaggle/working/model_versions/acoustic_classification4', exist_ok=True)
model.save(f'/kaggle/working/model_versions/acoustic_classification6.keras')
joblib.dump(scaler, "/kaggle/working/model_versions/scaler") 

['/kaggle/working/model_versions/scaler']

In [13]:
from IPython.display import FileLink
#!cd /kaggle/working/model_versions/
!ls
!zip -r file.zip model_versions
FileLink('file.zip')

data.csv  logs	model_versions
  adding: model_versions/ (stored 0%)
  adding: model_versions/acoustic_classification6.keras (deflated 8%)
  adding: model_versions/scaler (deflated 12%)


/kaggle/working/file.zip